In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Advanced Recording Example

Below we demonstrate a more advanced example of case recording including the four different objects
that a recorder can be attached to. We will then show how to extract various data from the model and finally,
relate that back to an XDSM for illustrative purposes.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_SellarMDAWithUnits", get_code("openmdao.test_suite.components.sellar_feature.SellarMDAWithUnits"), display=False)

:::{Admonition} `SellarMDAWithUnits` class definition 
:class: dropdown

{glue:}`code_SellarMDAWithUnits`
:::

In [ ]:
import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDAWithUnits

# build the model
prob = om.Problem(model=SellarMDAWithUnits())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

# setup the optimization
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

# Here we show how to attach recorders to each of the four objects:
#   problem, driver, solver, and system

# Create a recorder
recorder = om.SqliteRecorder('cases.sql')

# Attach recorder to the problem
prob.add_recorder(recorder)

# Attach recorder to the driver
prob.driver.add_recorder(recorder)

# To attach a recorder to a subsystem or solver, you need to call `setup`
# first so that the model hierarchy has been generated
prob.setup()

# Attach recorder to a subsystem
model.obj_cmp.add_recorder(recorder)

# Attach recorder to a solver
model.cycle.nonlinear_solver.add_recorder(recorder)

prob.set_solver_print(0)
prob.run_driver()
prob.record("final_state")
prob.cleanup()

The following XDSM diagram shows the SellarMDA component equations and their inputs and outputs. Through
the different recorders we can access the different parts of the model. We'll take you through an
example of each object and relate it back to the diagram.

![sellar_xdsm](images/sellar_xdsm.jpg)

## Driver
If we want to view the convergence of the model, the best place to find that by looking at the cases 
recorded by the `driver`. By default, a recorder attached to a driver will record the design variables,
constraints and objectives, so we will print them for the model at the end of the optimization. We'll 
use the helper methods like `get_objectives`, `get_design_vars`, `get_constraints` to return the info 
we're seeking.


In [ ]:
import openmdao.api as om

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# List driver cases (do not recurse to system/solver cases)
driver_cases = cr.list_cases('driver', recurse=False);
last_case = cr.get_case(driver_cases[-1])

objectives = last_case.get_objectives()
design_vars = last_case.get_design_vars()
constraints = last_case.get_constraints()

print(objectives['obj'])
print(design_vars['x'])
print(design_vars['z'])
print(constraints['con1'])
print(constraints['con2'])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(objectives['obj'], 3.18339395, 1e-8)
assert_near_equal(design_vars['x'], 0., 1e-8)
assert_near_equal(design_vars['z'], [1.97763888, 1.25035459e-15], 1e-8)
assert_near_equal(constraints['con1'], -1.68550507e-10, 1e-8)
assert_near_equal(constraints['con2'], -20.24472223, 1e-8)

## Plotting Design Variables

When inspecting or debugging a model, it can be helpful to visualize the path of the design
variables to their final values. To do this, we can list the cases of the driver and plot the data
with respect to the iteration number.

In [ ]:
import matplotlib.pyplot as plt
import openmdao.api as om

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# Get driver cases (do not recurse to system/solver cases)
driver_cases = cr.get_cases('driver', recurse=False)

# Plot the path the design variables took to convergence
# Note that there are two lines in the right plot because "Z"
# contains two variables that are being optimized
dv_x_values = []
dv_z_values = []
for case in driver_cases:
    dv_x_values.append(case['x'])
    dv_z_values.append(case['z'])

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)

ax1.plot(np.arange(len(dv_x_values)), np.array(dv_x_values))
ax1.set(xlabel='Iterations', ylabel='Design Var: X', title='Optimization History')
ax1.grid()

ax2.plot(np.arange(len(dv_z_values)), np.array(dv_z_values))
ax2.set(xlabel='Iterations', ylabel='Design Var: Z', title='Optimization History')
ax2.grid()

plt.show()

## Problem

A `Problem` recorder is best if you want to record an arbitrary case before or after a running the
model. In our case, we have placed our point at the end of the model.

In [ ]:
import openmdao.api as om

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# get list of cases recorded on problem
problem_cases = cr.list_cases('problem')

# get list of variables recorded on problem
problem_vars = cr.list_source_vars('problem')
print(problem_vars['outputs'])

# get the recorded case and check values
case = cr.get_case('final_state')
print(case.get_design_vars())
print(case.get_constraints())
print(case.get_objectives())

In [ ]:
assert(problem_cases == ['final_state'])
assert(sorted(problem_vars['outputs']) == ['con1', 'con2', 'obj', 'x', 'y1', 'y2', 'z'])
assert_near_equal(case.get_design_vars(), {'x': 0., 'z': [1.9776, 1.25e-15]}, tolerance=1e-4)
assert_near_equal(case.get_constraints(), {'con1': 0., 'con2': -20.2447}, tolerance=1e-4)
assert_near_equal(case.get_objectives(), {'obj': 3.18339395}, tolerance=1e-4)

## System
Suppose we want to know the values of the inputs to the objective function. 
For this we will use the `system` recorder. Using the `list_cases` method, we'll get the list of 
cases that were recorded by the objective component `root.obj_comp`. Then we use `get_case` to 
get each case and determine the input names using the keys of the case's inputs dictionary. 
Since we want to find the values going into the objective function for each execution, 
we iterate through those 14 cases and get the values in each.

In [ ]:
import numpy as np

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# Get a list of cases for the objective component
system_cases = cr.list_cases('root.obj_cmp', recurse=False)

# Number of cases recorded for 'obj_cmp'
print(f"Number of cases: {len(system_cases)}")

for case_num, case_id in enumerate(system_cases):
    case = cr.get_case(case_id)
    
    # Get the names of all the inputs to the objective component
    inputs = case.inputs.keys()
    
    # Get the rounded value of each input (as a python list)
    values = [(name, case[name].round(10).tolist()) for name in inputs]
    
    # Print the input values for this case
    print(f"{case_num:>2}: {values}")

In [ ]:
assert(sorted(inputs) == ['x', 'y1', 'y2', 'z'])

assert(len(system_cases) == 14), f"Expected 14 cases, but got {len(system_cases)}"

assert_near_equal([case['y1'].item() for case in cr.get_cases('root.obj_cmp')],
                  [25.6, 25.6, 8.33, 4.17, 3.30, 3.18, 3.16,
                   3.16, 3.16, 3.16, 3.16, 3.16, 3.16, 3.16],
                  tolerance=1e-1)

## Solver

Similar to the `system` recorder, we can query the cases recorded by the `solver`. You can also
access the values of inputs to the equation with the solver but in this case we'll focus on the
coupling variables `y1` and `y2`.

We'll use `list_cases` to get the cases recorded for `'root.cycle.nonlinear_solver'`, find out how many
cases there are and plot a subset of the solver iterations (covering multiple driver iterations)
to see the final convergence.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import openmdao.api as om

# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# Get list of cases, without displaying them
solver_cases = cr.list_cases('root.cycle.nonlinear_solver', out_stream=None)

# we will plot the last half of the iterations
num_cases = len(solver_cases)
sub_cases = num_cases//2
iterations = np.arange(num_cases-sub_cases, num_cases)

# Plot the final convergence of the two coupling variables
y1_history = []
y2_history = []

for case_id in solver_cases[-sub_cases:]:
    case = cr.get_case(case_id)
    y1_history.append(case['y1'])
    y2_history.append(case['y2'])

fig, (ax1, ax2) = plt.subplots(2, 1)

ax1.plot(iterations, np.array(y1_history))
ax1.set(ylabel='Coupling Output: y1', title='Solver History')
ax1.grid()

ax2.plot(iterations, np.array(y2_history))
ax2.set(ylabel='Coupling Parameter: y2', xlabel='Iterations')
ax2.grid()

plt.show()

# Get the final values
print(f"Number of cases: {len(solver_cases)}")
print("Final values:")
case = cr.get_case(solver_cases[-1])
print('  y1:', case['y1'])
print('  y2:', case['y2'])

In [ ]:
assert(len(solver_cases) == 76)
assert_near_equal(case['y1'], 3.16, 1e-8)
assert_near_equal(case['y2'], 3.75527777, 1e-8)